In [1]:
#The get_execution_role function retrieves the AWS Identity and Access Management (IAM) role you created at the time of creating your notebook instance.

import boto3, re
from sagemaker import get_execution_role
from keras.models import model_from_json

role = get_execution_role()

Using TensorFlow backend.


In [2]:
!pip freeze

absl-py==0.5.0
alabaster==0.7.10
anaconda-client==1.6.14
anaconda-project==0.8.2
asn1crypto==0.24.0
astor==0.7.1
astroid==1.6.3
astropy==3.0.2
attrs==18.1.0
Automat==0.3.0
autovizwidget==0.12.5
awscli==1.16.79
Babel==2.5.3
backcall==0.1.0
backports.shutil-get-terminal-size==1.0.0
beautifulsoup4==4.6.0
bitarray==0.8.1
bkcharts==0.2
blaze==0.11.3
bleach==2.1.3
bokeh==0.12.13
boto==2.48.0
boto3==1.9.69
botocore==1.12.69
Bottleneck==1.2.1
cached-property==1.5.1
certifi==2018.8.24
cffi==1.11.5
characteristic==14.3.0
chardet==3.0.4
click==6.7
cloudpickle==0.5.3
clyent==1.2.2
colorama==0.3.9
contextlib2==0.5.5
cryptography==2.2.2
cycler==0.10.0
Cython==0.28.4
cytoolz==0.9.0.1
dask==0.17.5
datashape==0.5.4
decorator==4.3.0
distributed==1.21.8
docker==3.6.0
docker-compose==1.23.2
docker-pycreds==0.4.0
dockerpty==0.4.1
docopt==0.6.2
docutils==0.14
entrypoints==0.2.3
environment-kernels==1.1.1
et-xmlfile==1.0.1
fastcache==1.0.2
filelock==3.0.4
Flask==1.0.2
Flask-Cors==3.0.4
gast==0.2.0
gevent==1.

In [3]:
#Your trained model will need to be saved in either the Keras (JSON and weights hdf5) format
#Load the Keras model using the JSON and weights file

import keras


#make a folder for the trained model. then upload the (model.json and model-weights.h5) files there.
#UPLOAD NOT USING CODE. USING THE UPLOAD BUTTON ON JUPYTER
#!mkdir keras_model


In [4]:
json_file = open('keras_model/model.json', 'r')
loaded_model_json = json_file.read()
#json_file.close()

In [5]:
from keras.models import model_from_json
loaded_model = model_from_json(loaded_model_json)

In [6]:
loaded_model.load_weights('keras_model/model.h5')

In [7]:
print("Loaded model from disk")

Loaded model from disk


In [8]:
from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants

In [9]:
model_version = '1'
export_dir = 'export/Servo/' + model_version

In [10]:
builder = builder.SavedModelBuilder(export_dir)

In [11]:
signature = predict_signature_def(
    inputs={"inputs": loaded_model.input}, outputs={"score": loaded_model.output})

In [12]:
from keras import backend as K

with K.get_session() as sess:
    # Save the meta graph and variables
    builder.add_meta_graph_and_variables(
        sess=sess, tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
    builder.save()

In [13]:
!ls export

Servo


In [14]:
!ls export/Servo

1


In [15]:
!ls export/Servo/1

saved_model.pb	variables


In [16]:
!ls export/Servo/1/variables

variables.data-00000-of-00001  variables.index


In [17]:
import tarfile
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)

In [18]:
import sagemaker

sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='model')

In [19]:
!touch train.py

In [20]:
from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/model/model.tar.gz',
                                  role = role,
                                  entry_point = 'train.py')

In [21]:
%%time
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-tensorflow-2019-02-14-03-17-27-380
INFO:sagemaker:Creating endpoint with name sagemaker-tensorflow-2019-02-14-03-17-27-380


---------------------------------------------------------------!CPU times: user 444 ms, sys: 11.8 ms, total: 455 ms
Wall time: 5min 21s


In [22]:
endpoint_name = 'sagemaker-tensorflow-2019-02-14-03-17-27-380'

In [23]:
import sagemaker
from sagemaker.tensorflow.model import TensorFlowModel
predictor=sagemaker.tensorflow.model.TensorFlowPredictor(endpoint_name, sagemaker_session)

In [38]:
import numpy as np
from PIL import Image
import json
import cv2
import base64

# The sample model expects an input of shape [1,50]
data = cv2.imread('01d7124bf7f7410e9c20c2c0705ce0e1.jpg')
data = base64.b64encode(data)

In [46]:
image_data = json.dumps({'picture' : str(data)})
predictor.predict(image_data)

ConnectionClosedError: Connection was closed before we received a valid response from endpoint URL: "https://runtime.sagemaker.ap-southeast-1.amazonaws.com/endpoints/sagemaker-tensorflow-2019-02-14-03-17-27-380/invocations".